# Compute equilibrium
I provide routines to evaluate useful variables: values $U_i$ and $W_{ij}$, and after-tax wages $w_{ij}$. Wages satisfy
\begin{align}
    w_{ij}&=(r+\delta_j)\tilde{\beta}(X_{ij}-T_{ij})+h+\int_\mathcal{J} \left[s(e^0_{ij_1})m(\theta_{j_1})\tilde\beta (X_{ij_1}-T_{ij_1})- e^0_{ij_1}\right]dj_1- \int_\mathcal{J} \left[\xi s(e^{j_0}_{ij_1})m(\theta_{j_1})\tilde\beta (X_{ij_1}-X_{ij_0}-TT^{j_0}_{ij_1})- e^{j_0}_{ij_1}\right]dj_1\\
    w_{ij}&=(r+\delta_j)\tilde{\beta}(X_{ij}-T_{ij})+y_{ij}-(r+\delta_j)X_{ij}
\end{align}

In [ ]:
function Uval(i, A::Alloc,T::Taxes,M::Para,N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)))
    out = M.h
    for j in 1:N.J  
        bXT = betatilde * ( A.X[i,j] - T_tax(i,j,A.X,T,M))
        out += s(A.e0[i,j],M) * m(A.theta[j],M) * bXT - A.e0[i,j]
    end
    out /= M.r
    return(out)
end

function wage(i,j0, A::Alloc,T::Taxes,M::Para,N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w))) 
    
    out = (M.r+M.delta[j0]) * betatilde * (A.X[i,j0] - T_tax(i,j0,A.X,T,M))
    out += M.y[i,j0] - (M.r+M.delta[j0]) * (A.X[i,j0])
    return(  out ) 
end

## 1) Compute a segmented equilibrium
We compute an equilibrium in a model where workers of type $i$ can only obtain a job $j=i$. In particular, there is no OTJ search.
For each segment $j$,
\begin{align}
    & (r+\delta_j)X_{jj} = y_{jj}-h-s(e^0_{jj})m(\theta_{j})\tilde\beta (X_{jj}-T_{jj})+ e^0_{jj}\\
    & 0=\frac{k_{j}}{q_j(\theta_{j})}-(1-\tilde\beta) (X_{jj}-T_{jj})
\end{align}
with
\begin{align*}
    & n_{jj} = \frac{s(e^0_{jj})m(\theta_{j})}{\delta_j+s(e^0_{jj})m_j(\theta_j)}  \nu_j
\end{align*}

In [ ]:
function objective_segm_j(j,Xjj,thetaj, T::Taxes,M::Para,N::Hyper;
    betatilde=M.beta / (M.beta + (1-M.beta)*(1+T.w)) )

    Tjj = T_tax_sca(j,M.y[j,j],Xjj,T,M)
    bXT = betatilde * ( Xjj-Tjj )
    ret = r2s( m(thetaj,M) * bXT, M)

    err1 = -(M.r+M.delta[j])*Xjj + M.y[j,j] - M.h - ret
    err2 = M.k[j]/q(thetaj,M) - (1-betatilde) * (Xjj-Tjj)
    return(err1^2 + err2^2)
end
    
function compute_segm_j(j,T::Taxes,M::Para,N::Hyper;
        vec0 = [(M.y[j,j]-M.h)/(M.r+M.delta[j]) , 1] ,
        betatilde=M.beta / (M.beta + (1-M.beta)*(1+T.w)) )
    
    opt = Opt(:LN_BOBYQA, 2) ## :LN_NELDERMEAD :LN_SBPLX :LN_BOBYQA :LN_COBYLA
   # xtol_rel!(opt,1e-5)
    lower_bounds!(opt, [0, 1e-5])
    upper_bounds!(opt, [Inf,Inf])
    
    obj(vec,g) = objective_segm_j(j,vec[1],vec[2], T,M,N, betatilde=betatilde)
    min_objective!(opt,obj)

    (minf,vec,ret) = optimize(opt,vec0)
 #   println("The minimum is reached at ",minf, " with ", ret)

    return(vec)
end

function compute_segm_equilibrium(T::Taxes,M::Para,N::Hyper)
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w))
    
    A = Alloc(0.,N) 
    for j in 1:N.J
        (Xjj,thetaj) = compute_segm_j(j,T,M,N,betatilde=betatilde) 
        Tjj = T_tax_sca(j,M.y[j,j],Xjj,T,M)
        bXT = betatilde * ( Xjj-Tjj )
        ejj,r2s = search(m(thetaj,M) * bXT,M) 
        njj = s(ejj,M)*m(thetaj,M) /(M.delta[j]+s(ejj,M)*m(thetaj,M)) * M.nu[j]
        
        A.X[j,j] = Xjj
        A.e0[j,j] = ejj
        A.theta[j] = thetaj
        A.n[j,j] = njj
        A.u[j] = M.nu[j] - njj
        
        Uj = (M.h + r2s ) / M.r
        for j1 in 1:N.J
            A.X[j,j1] = (M.y[j,j1] + M.delta[j1] * Uj) / (M.r+M.delta[j1]) - Uj ## no OTJ
        end
    end
    return(A)
end


For the calibration, we need to find the segmented equilibrium for specifc $y_{jj}$ (diagonal only), $\theta_j$ and $\nu_i$ fixed.

In [ ]:
function conf_segm_equilibrium_theta(vec, M::Para,N::Hyper)
    ## vec = [ydiag,theta,nuc]
    ydiag = vec[1:N.J]
    theta = vec[N.J .+ (1:N.J)]
    nu = nuc2nu(vec[(N.J + N.J + 1):end])
    
    ## create new Para
    yout = copy(M.y)
    for j in 1:N.J
        yout[j,j] = ydiag[j]
    end
    Mout = Para(nu=nu,r=M.r,delta=M.delta,eta=M.eta,beta=M.beta,
        y=yout,h=M.h, k=M.k, es=M.es, xi=M.xi)
    return(Mout,theta)
end

function compute_segm_equilibrium_theta(vec,
        T::Taxes,M::Para,N::Hyper;
        betatilde=M.beta / (M.beta + (1-M.beta)*(1+T.w)) )
     ## vec = [ydiag,theta,nuc]
    
    M1, theta = conf_segm_equilibrium_theta(vec, M,N)

    ## find new allocation
    A1 = Alloc(0.,N) 
    A1.X[:] .= -10000.
    
    for j in 1:N.J
        function objective_Xjj(Xjj)
            Tjj = T_tax_sca(j,M1.y[j,j],Xjj,T,M1)
            bXT = betatilde * ( Xjj-Tjj )
            ret = r2s( m(theta[j],M1) * bXT, M1)
            err1 = -(M1.r+M1.delta[j])*Xjj + M1.y[j,j] - M1.h - ret
            return(err1)
        end
        Xjj = fzero(objective_Xjj,0.,1e5)
        Tjj = T_tax_sca(j,M1.y[j,j],Xjj,T,M1)
        bXT = betatilde * ( Xjj-Tjj )
        ejj = e2s( m(theta[j],M1) * bXT, M1)
        njj = s(ejj,M1)*m(theta[j],M1) /(M1.delta[j]+s(ejj,M1)*m(theta[j],M1)) * M1.nu[j]
        
        A1.X[j,j] = Xjj
        A1.theta[j] = theta[j]
        A1.e0[j,j] = ejj
        A1.n[j,j] = njj
        A1.u[j] = M1.nu[j] - njj
        
    end
    
    return(A1,M1)
end



## 2) Compute a full equilibrium
We proceed sequentially using the structure of the model.
1. We find the surpluses $X$ for a given vector $\theta$.
2. With $\theta$ and $X$, we find the optimal search efforts.
3. Then, we compute the distribution of unemployed and employed workers.
4. Lastly, we wrap the three previous step into a function and we find $\theta$ as a fixed point.

Step 1.
We solve for the surpluses $X$ using the contraction mapping $X_{t+1} = X_t + step. F(X_t|\theta) $.
$$F(X|\theta)_{ij_0}=-(r+\delta_{j_0})X_{ij_0} + y_{ij_0}-h-\int_\mathcal{J} \left[s(e^0_{ij_1})m(\theta_{j_1})\tilde\beta (X_{ij_1}-T_{ij_1})- e^0_{ij_1}\right]dj_1+ \int_\mathcal{J} \left[\xi s(e^{j_0}_{ij_1})m(\theta_{j_1})\tilde\beta (X_{ij_1}-X_{ij_0}-TT^{j_0}_{ij_1})- e^{j_0}_{ij_1}\right]dj_1
,$$
with the optimal efforts defined by their first-order conditions.
We make sure that $step$ is low enough to have convergence.
If we obtain NaN, then we use a minimisation algorithm.

In [ ]:
## fast method
function differential_X(X, theta, T::Taxes, M::Para, N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)))
    
    Xdiff = copy(M.y)
    for i in 1:N.I, j0 in 1:N.J
        Xdiff[i,j0] -=  M.h + (M.r+M.delta[j0]) * X[i,j0]
        for j1 in 1:N.J
            bXT = betatilde * ( X[i,j1] - T_tax(i,j1,X,T,M))
            mj1 = m(theta[j1],M)
            Xdiff[i,j0] -= r2s(mj1 * bXT, M)
              
            bXXT = betatilde * (X[i,j1] - X[i,j0] - TT_tax(i,j0,j1,X,T,M))
            Xdiff[i,j0] += r2s(M.xi * mj1 * bXXT,M)
        end 
    end

    return(Xdiff, sqrt(sum(abs2,Xdiff)))
end

function compute_X_fast(theta, T::Taxes, M::Para, N::Hyper;
    X0=zeros(N.I,N.J), ## note: algorithm can diverge when X0!=0 is provided
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)),
    Xtol=1e-3, maxloop = 1e7, step = 0.2)

    X = copy(X0)
    dist = 1000.
    counter = 0
    while (dist > Xtol) & (counter<maxloop)
        Xdiff, dist = differential_X(X, theta, T, M, N,betatilde=betatilde)
        X += step .* Xdiff 
        counter += 1
    end
    if counter < maxloop
        return(X)
    else
        print(" - Maxloop reached, dist=", dist ," ")
        return( fill(NaN,N.I,N.J))
    end
end

#############################################################
## slow method
function differential_Xi(i, Xi, theta, T::Taxes, M::Para, N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)))
    
    Xdiff = copy(M.y[i,:])
    for j0 in 1:N.J
        Xdiff[j0] -=  M.h + (M.r+M.delta[j0]) * Xi[j0]
        for j1 in 1:N.J
            bXT = betatilde * ( Xi[j1] - T_tax_sca(j1,M.y[i,j1],Xi[j1],T,M))
            mj1 = m(theta[j1],M)
            Xdiff[j0] -= r2s(mj1 * bXT, M)
              
            bXXT = betatilde * (Xi[j1] - Xi[j0] - TT_tax_vec(j0,j1,M.y[i,:],Xi,T,M))
            Xdiff[j0] += r2s(M.xi * mj1 * bXXT,M)
        end 
    end

    return(sqrt(sum(abs2,Xdiff))) #Xdiff, sqrt(sum(abs2,Xdiff))
end

function compute_Xi(i, theta, T::Taxes, M::Para, N::Hyper; 
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)),
    Xi0=zeros(N.J),stop=1e-3)
    
    opt = Opt(:LN_BOBYQA, N.J) 
    #maxtime!(opt,60)
    stopval!(opt, stop)

    function obj(vec,g) 
        out = differential_Xi(i, vec, theta, T, M, N,betatilde=betatilde)
        #verbose ? print(out," ") : ()
        return(out)
    end
    min_objective!(opt,obj)

    (minf,Xi,ret) = optimize(opt,Xi0)
    #println("--The minimum is reached at ",minf, " with ", ret)
    
    return(Xi)
end

function compute_X_slow(theta, T::Taxes, M::Para, N::Hyper;
    X0=zeros(N.I,N.J),
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)),
    stop=1e-3)

    X = copy(X0)
    for i in 1:N.I
        X[i,:] = compute_Xi(i, theta, T, M, N,
            betatilde = betatilde, Xi0=X0[i,:], stop=stop)
    end
    return(X)
end

######################
### wrap-up
function compute_X(theta, T::Taxes, M::Para, N::Hyper;
    X0=zeros(N.I,N.J),
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)),
    Xtol=1e-2)
    
    X = compute_X_fast(theta, T, M, N, X0=X0, betatilde=betatilde, 
            Xtol=Xtol, maxloop = 1e7, step = 0.2)

    if any(isnan,X) 
        X = compute_X_slow(theta, T, M, N, X0=X0, betatilde=betatilde,stop=Xtol/N.J)
    end
    return(X)
end

Step2. Compute the optimal search efforts given $X$ and $\theta$.

In [ ]:
function compute_e(X, theta, T::Taxes,M::Para,N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)))
    
    e0 = zeros(N.I,N.J)
    e1 = zeros(N.I,N.J,N.J)
    for i in 1:N.I, j1 in 1:N.J
        mj1 = m(theta[j1],M)
        bXT = betatilde * ( X[i,j1] - T_tax(i,j1,X,T,M))
        e0[i,j1] = e2s(mj1 * bXT ,M)
        for j0 in 1:N.J
            bXXT = betatilde * (X[i,j1] - X[i,j0] - TT_tax(i,j0,j1,X,T,M))
            e1[i,j0,j1] = e2s(M.xi * mj1 * bXXT ,M) 
        end
    end
    return(e0,e1)
end

Step 3. Given search efforts, surplus and market tightness, we solve for the stock of employed and unemployed workers.
\begin{align*}
    & 0= s(e^0_{ij_0})m(\theta_{j_0})\left(\nu_i-\int_{\mathcal J}n_{ij_1}dj_1\right)+\int_{\mathcal{J}}\xi s(e_{ij_0}^{j_1})m(\theta_{j_0})n_{ij_1}dj_1 -\delta_{j_0} n_{ij_0}-\int_{\mathcal{J}}\xi s(e_{ij_1}^{j_0})m(\theta_{j_1})n_{ij_0}dj_1\\
    & u_i = \nu_i-\int_{\mathcal J}n_{ij_1}dj_1
\end{align*}

In [ ]:
function compute_n(e0, e1, X, theta, M::Para, N::Hyper)
    u = Vector{Float64}(undef,N.I)
    n = Array{Float64}(undef,N.I,N.J)
    Bvec = Vector{Float64}(undef,N.J)
    
    for i in 1:N.I
        Amat = zeros(N.J,N.J)
        for j0 in 1:N.J 
            Bvec[j0] = s(e0[i,j0],M) * m(theta[j0],M) * M.nu[i]
            for j1 in 1:N.J
                Amat[j0,j1] += s(e0[i,j0],M) * m(theta[j0],M)
                Amat[j0,j1] -= M.xi * s(e1[i,j1,j0],M) * m(theta[j0],M)
                Amat[j0,j0] += M.xi * s(e1[i,j0,j1],M) * m(theta[j1],M)
            end
            Amat[j0,j0] += M.delta[j0]
        end
        n[i,:] = max.(Amat \  Bvec , 0.) ## numerical errors can generate low negative numbers like -1e-16
        u[i] = M.nu[i]-sum(n[i,:])
    end
    
    return(u,n)
end
    

Step 4. We can finally use the equilibrium conditions of $\theta$ to define an objective function
\begin{align*}
     0 &=\int_{\mathcal I\times\mathcal J}\xi s(e_{ij_1}^{j_0})\left(\frac{k_{j_1}}{q(\theta_{j_1})}-(1-\tilde\beta) (X_{ij_1}-X_{ij_0}-TT^{j_0}_{ij_1})\right)n_{ij_0}didj_0
     +\int_\mathcal{I}s(e_{ij_1}^{0})\left(\frac{k_{j_1}}{q(\theta_{j_1})}-(1-\tilde\beta) (X_{ij_1}-T_{ij_1})\right) u_idi
\end{align*}
Note we do ont use this equilibrium condition when we calibrate our model.

In [ ]:
function objective_theta(u, n, e0, e1, X, theta, T::Taxes, M::Para, N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)))

    res = zeros(N.J)
    for j1 in 1:N.J
        kq = M.k[j1] / q(theta[j1],M)
        for i in 1:N.I
            ## p = 0
            kqX = kq - (1-betatilde) * ( X[i,j1] - T_tax(i,j1,X,T,M))
            res[j1] += s(e0[i,j1],M) * kqX * u[i]
            
            ## p =j0
            for j0 in 1:N.J
                kqX = kq - (1-betatilde) * (X[i,j1] - X[i,j0] - TT_tax(i,j0,j1,X,T,M))
                res[j1] += M.xi * s(e1[i,j0,j1],M) * n[i,j0] * kqX
            end
        end
    end 
    return(sqrt(sum(abs2,res)) / N.J)
end


function compute_equilibrium(T::Taxes,M::Para,N::Hyper ;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)),
    theta0 = fill(0.1,N.J),  #compute_segm_equilibrium(T,M,N).theta
    verbose = false, maxti= 60, Xtol=1e-2, algo=:LN_BOBYQA, valf = 1e-3 )   

    ## parameters for optimization
    opt = Opt(algo, length(theta0)) 
  #  xtol_rel!(opt,1e-10)
    stopval!(opt,valf)
    maxtime!(opt,maxti)
    
    lower_bounds!(opt, fill(1e-15,N.J))
    upper_bounds!(opt, fill(Inf,N.J))
    
    function obj(theta,g) 
        X = compute_X(theta, T,M,N,betatilde=betatilde, Xtol=Xtol)
        e0, e1 = compute_e(X,theta, T,M,N,betatilde=betatilde)
        u, n = compute_n(e0, e1, X, theta,M,N)
        out = objective_theta(u, n, e0, e1, X, theta, T,M,N,betatilde=betatilde)
        verbose ? print(out, " ") : ()
        return(out)
    end
    min_objective!(opt,obj)

    (minf,theta,ret) =  optimize(opt,theta0)     
    println("The minimum is reached at ",minf, " with ", ret) 
    
    ### alloc output
    X = compute_X(theta, T,M,N,betatilde=betatilde,Xtol=Xtol)
    e0, e1 = compute_e(X,theta, T,M,N,betatilde=betatilde)
    u, n = compute_n(e0, e1, X, theta,M,N)
    return(Alloc(X,e0,e1,theta,u,n))
end


The following function proposes another way to solve for the equilibrium using all the equations:
\begin{align*}
     & (r+\delta_{j_0})X_{ij_0} = y_{ij_0}-h_i-\int_\mathcal{J} \left[s(e^0_{ij_1})m(\theta_{j_1})\tilde\beta (X_{ij_1}-T_{ij_1})- e^0_{ij_1}\right]dj_1+ \int_\mathcal{J} \left[\xi s(e^{j_0}_{ij_1})m(\theta_{j_1})\tilde\beta (X_{ij_1}-X_{ij_0}-TT^{j_0}_{ij_1})- e^{j_0}_{ij_1}\right]dj_1 \\
     & s'(e^0_{ij_1})m(\theta_{j_1})=max(1/(\tilde\beta (X_{ij_1}-T_{ij_1})),0)\\
     & s'(e^{j_0}_{ij_1})m(\theta_{j_1})=max(1/(\tilde\beta (X_{ij_1}-X_{ij_0}-T_{ij_1})),0)\\
     0 &=\int_{\mathcal I\times\mathcal J}\xi s(e_{ij_1}^{j_0})\left(\frac{k_{j_1}}{q(\theta_{j_1})}-(1-\tilde\beta) (X_{ij_1}-X_{ij_0}-TT^{j_0}_{ij_1})\right)n_{ij_0}didj_0
     +\int_\mathcal{I}s(e_{ij_1}^{0})\left(\frac{k_{j_1}}{q(\theta_{j_1})}-(1-\tilde\beta) (X_{ij_1}-T_{ij_1})\right) u_idi\\
    & 0= s(e^0_{ij_0})m(\theta_{j_0})u_i+\int_{\mathcal{J}}\xi s(e_{ij_0}^{j_1})m(\theta_{j_0})n_{ij_1}dj_1 -\delta_{j_0} n_{ij_0}-\int_{\mathcal{J}}\xi s(e_{ij_1}^{j_0})m(\theta_{j_1})n_{ij_0}dj_1\\
    & u_i = \nu_i-\int_{\mathcal J}n_{ij_1}dj_1
\end{align*}

In [ ]:
function objective_equilibrium(A::Alloc,T::Taxes,M::Para,N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)))
    
    eqres = Alloc(0.,N) 
    
    ## X[i,j0] and e0, e1
    for i in 1:N.I, j0 in 1:N.J
        eqres.X[i,j0] = -(M.r+M.delta[j0]) * A.X[i,j0]
        eqres.X[i,j0] += M.y[i,j0] - M.h
        for j1 in 1:N.J
            bXT = betatilde * ( A.X[i,j1] - T_tax(i,j1,A.X,T,M))
            mj1 = m(A.theta[j1],M)
            eqres.X[i,j0] -= s(A.e0[i,j1],M) * mj1 * bXT  - A.e0[i,j1]
            if j0 == 1
                eqres.e0[i,j1] =  A.e0[i,j1] - e2s(mj1*bXT,M) 
            end
            bXXT = betatilde * (A.X[i,j1] - A.X[i,j0] - TT_tax(i,j0,j1,A.X,T,M))
            eqres.X[i,j0] += M.xi * s(A.e1[i,j0,j1],M) * mj1 * bXXT - A.e1[i,j0,j1]
            eqres.e1[i,j0,j1] =  A.e1[i,j0,j1] - e2s(M.xi*mj1*bXXT,M) 
        end
    end    
    
    ## theta[j1]
    for j1 in 1:N.J
        kq = M.k[j1] / q(A.theta[j1],M)
        for i in 1:N.I
            kqX = kq - (1-betatilde) * ( A.X[i,j1] - T_tax(i,j1,A.X,T,M))
            eqres.theta[j1] += s(A.e0[i,j1],M) * kqX * A.u[i]

            for j0 in 1:N.J
                kqX = kq - (1-betatilde) * (A.X[i,j1] - A.X[i,j0] - TT_tax(i,j0,j1,A.X,T,M))
                eqres.theta[j1] += M.xi * s(A.e1[i,j0,j1],M) * A.n[i,j0] * kqX
            end
        end
    end
    
    ## n[i,j0] and u[i]
    for i in 1:N.I
        eqres.u[i] = A.u[i] - M.nu[i]
        for j0 in 1:N.J
            eqres.u[i] += A.n[i,j0]
        
            se0 = s(A.e0[i,j0],M)
            mj0 = m(A.theta[j0],M)
            eqres.n[i,j0] = se0 * mj0 * A.u[i] - M.delta[j0] * A.n[i,j0]

            for j1 in 1:N.J
                eqres.n[i,j0] += M.xi * s(A.e1[i,j1,j0],M) * mj0 * A.n[i,j1] 
                mj1 = m(A.theta[j1],M)
                eqres.n[i,j0] -= M.xi * s(A.e1[i,j0,j1],M) * mj1 * A.n[i,j0]
            end
        end
    end  
    d1 = sum(abs2,eqres.X)
    d2 = sum(abs2,eqres.e0)
    d3 = sum(abs2,eqres.e1)
    d4 = sum(abs2,eqres.theta)
    d5 = sum(abs2,eqres.u)
    d6 = sum(abs2,eqres.n)
    println("distances: $d1, $d2, $d3, $d4, $d5, $d6.")
    return(d1+d2+d3+d4+d5+d6) 
end

#=
function compute_equilibrium2(T::Taxes,M::Para,N::Hyper ; 
        verbose = false, valf = 1e-10, maxti= 60,
        A0 = segm_alloc(T,M,N))    

    ## parameters for optimization
    Npar = N.I*N.J + N.I*N.J + N.I*N.J*N.J + N.J + N.I + N.I*N.J
    opt = Opt(:LN_BOBYQA, Npar) ## :LD_MMA :LD_LBFGS :LN_NELDERMEAD :LN_SBPLX :LN_BOBYQA :LN_COBYLA
   # xtol_rel!(opt,1e-5)
    stopval!(opt,valf)
    maxtime!(opt,maxti)
    
    lower_bounds!(opt, vectorise(lbd_alloc(N),N))
    upper_bounds!(opt, vectorise(ubd_alloc(N),N))
    
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w))
    function obj(vec,g) 
        out = objective_equilibrium(allocise(vec,N),T,M,N,betatilde=betatilde)
        return(out)
    end
    min_objective!(opt,obj)

    vec0 = vectorise(A0,N)
    (minf,vec,ret) =  optimize(opt,vec0)
    verbose ? println("The minimum is reached at ",minf, " with ", ret) : ()
    
    return(allocise(vec,N), minf)
end=#

For the calibration, we need to find the equilibrium for a vector of $y_{jj}$, $\alpha_j$, $\epsilon$, $\xi$ and $\theta_j$ fixed.

In [ ]:
function conf_equilibrium_theta(vec, M::Para,N::Hyper)
    ## vec=[yhigh,alpha,es,xi,theta,nuc]
    ydiag = vec[1:N.J]
    alpha = vec[N.J .+ (1:N.J)]
    y = build_y(ydiag,alpha, M.h)
    es = vec[2*N.J+1]
    xi = vec[2*N.J+2]
    theta = vec[2*N.J+2 .+ (1:N.J)]
    nu = nuc2nu(vec[(3*N.J+3):end])
    
    ## create new Para
    Mout = Para(nu=nu,r=M.r,delta=M.delta,eta=M.eta,beta=M.beta,
        y=y,h=M.h, k=M.k, es=es, xi=xi)
    return(Mout,theta)
end

function compute_equilibrium_theta(vec,T::Taxes,M::Para,N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)),
    Xtol=1e-2)
    ## vec=[yhigh,alpha,es,xi,theta,nuc]
    
    M1, theta = conf_equilibrium_theta(vec, M,N)
    
    ## new alloc
    X = compute_X(theta, T,M1,N,betatilde=betatilde, Xtol=Xtol) #X0=copy(A0.X) with A0 = compute_segm_equilibrium(T,M,N)
    e0, e1 = compute_e(X,theta, T,M1,N,betatilde=betatilde)
    u, n = compute_n(e0, e1, X, theta,M1 ,N)
    A1 = Alloc(X,e0,e1,theta,u,n) 
    return(A1,M1)
end


## 3) Welfare and optimum
The welfare writes $\int_0^{\infty} Y_t \exp(-rt)dt$, with
\begin{align}
Y_t&=\int_{\mathcal{I}}rU_i\nu_idi +\int_{\mathcal{I}\times\mathcal{J}} rW_{ij}n_{ij}didj+\int_{\mathcal{I}\times\mathcal{J}} rT_{ij}n_{ij}didj\\
&= \int_{\mathcal{I}}rU_i\nu_i + \int_{\mathcal{I}\times\mathcal{J}} r\tilde{\beta}(X_{ij}-T_{ij})n_{ij}didj+\int_{\mathcal{I}\times\mathcal{J}} rT_{ij}n_{ij}didj.
\end{align}
Note: this formula excludes the case $\Theta_{ij_1}^{j0}\neq 0$.


In [ ]:
## welfare for decentralized equilibrium
function welfare_dec(A::Alloc,T::Taxes,M::Para,N::Hyper;
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w)))
    
    welf = 0.
    for i = 1:N.I
        inc =  M.r * Uval(i,A,T,M,N,betatilde=betatilde) * M.nu[i]
        
        for j1 in 1:N.J
            inc += M.r * betatilde * (A.X[i,j1]-T_tax(i,j1,A.X,T,M)) * A.n[i,j1]
            inc += M.r * T_tax(i,j1,A.X,T,M) * A.n[i,j1]
        end
        
        welf +=  inc
    end
    return(welf)
end

## Uval for opti
function Uval_opt(i, A::Alloc,M::Para,N::Hyper)
    out = M.h
    for j in 1:N.J  
        Xk = A.X[i,j] - M.k[j] / q(A.theta[j],M)
        out += s(A.e0[i,j],M) * m(A.theta[j],M) * Xk - A.e0[i,j]
    end
    out /= M.r
    return(out)
end


## welfare for social opti
function welfare_opt(opA::Alloc,M::Para,N::Hyper)
    
    welf = 0.
    for i = 1:N.I
        inc =  M.r * Uval_opt(i,opA,M,N) * M.nu[i]
        
        for j1 in 1:N.J
            inc += M.r * (1-M.eta) * opA.X[i,j1] * opA.n[i,j1] ## no taxes since integral of taxes=0
        end
        
        welf +=  inc 
    end
    return(welf)
end


## other welfare
function welfare2(A::Alloc,M::Para,N::Hyper)
    welf = 0.
    for i in 1:N.I, j in 1:N.J
        welf += M.y[i,j] * A.n[i,j]
    end
    for i in 1:N.I
        welf += M.h * A.u[i]
    end
    for i in 1:N.I, j in 1:N.J
        welf -= M.k[j] * A.theta[j] * Opmismatch.s(A.e0[i,j],M) * A.u[i]
    end
    for i in 1:N.I, j0 in 1:N.J, j in 1:N.J
        welf -= M.k[j] * A.theta[j] * M.xi * Opmismatch.s(A.e1[i,j0,j],M) * A.n[i,j0]
    end
        for i in 1:N.I, j in 1:N.J
        welf -= A.e0[i,j] * A.n[i,j]
    end
        for i in 1:N.I, j0 in 1:N.J, j in 1:N.J
        welf -= A.e1[i,j0,j] * A.n[i,j0]
    end
    return(welf)
end


To determine the efficient allocation, we find the objective function to minimise using the same approach as the decentralised equilibrium.
\begin{align*}
    & F(X|\theta)_{ij_0} = -(r+\delta_{j_0})X_{ij_0} + y_{ij_0}-h-\int_\mathcal{J} \left[s(e^0_{ij_1})m(\theta_{j_1}) (X_{ij_1}-\frac{k_{j_1}}{q(\theta_{j_1})})- e^0_{ij_1}\right]dj_1+ \int_\mathcal{J} \left[\xi s(e^{j_0}_{ij_1})m(\theta_{j_1}) (X_{ij_1}-X_{ij_0}-\frac{k_{j_1}}{q(\theta_{j_1})})- e^{j_0}_{ij_1}\right]dj_1 
\end{align*}

In [ ]:
## fast method
function differential_X_opt(X, theta, M::Para, N::Hyper)
    
    Xdiff = copy(M.y)
    for i in 1:N.I, j0 in 1:N.J
        Xdiff[i,j0] -=  M.h + (M.r+M.delta[j0]) * X[i,j0]
        for j1 in 1:N.J
            Xk = X[i,j1] - M.k[j1] / q(theta[j1],M)
            mj1 = m(theta[j1],M)
            Xdiff[i,j0] -= r2s(mj1 * Xk, M)
              
            XXk = X[i,j1] - X[i,j0] - M.k[j1] / q(theta[j1],M)
            Xdiff[i,j0] += r2s(M.xi * mj1 * XXk,M)
        end 
    end

    return(Xdiff, sqrt(sum(abs2,Xdiff)))
end
 
function compute_X_opt_fast(theta, M::Para, N::Hyper;
    X0=zeros(N.I,N.J), ## note: algorithm can diverge when X0!=0 is provided
    Xtol=1e-3, maxloop = 1e7, step = 0.2)

    X = copy(X0)
    dist = 1000
    counter = 0
    while (dist > Xtol) & (counter<maxloop)
        Xdiff, dist = differential_X_opt(X, theta, M, N)
        X += step .* Xdiff 
        counter += 1
    end
    if counter < maxloop
        return(X)
    else
        print(" - Maxloop reached, dist=", dist ," ")
        return( fill(NaN,N.I,N.J))
    end
end


#############################################################
## slow method
function differential_Xi_opt(i, Xi, theta, M::Para, N::Hyper)
    
    Xdiff = copy(M.y[i,:])
    for j0 in 1:N.J
        Xdiff[j0] -=  M.h + (M.r+M.delta[j0]) * Xi[j0]
        for j1 in 1:N.J
            Xk = Xi[j1] - M.k[j1] / q(theta[j1],M)
            mj1 = m(theta[j1],M)
            Xdiff[j0] -= r2s(mj1 * Xk, M)
              
            XXk = Xi[j1] - Xi[j0] - M.k[j1] / q(theta[j1],M)
            Xdiff[j0] += r2s(M.xi * mj1 * XXk,M)
        end 
    end
    return(sqrt(sum(abs2,Xdiff))) #Xdiff, sqrt(sum(abs2,Xdiff))
end

function compute_Xi_opt(i, theta, M::Para, N::Hyper; 
    Xi0=zeros(N.J),stop=1e-3)
    
    opt = Opt(:LN_BOBYQA, N.J) 
    #maxtime!(opt,60)
    stopval!(opt, stop)

    Xlbd = fill(-Inf, N.J)
    Xlbd[i] = 0.
    lower_bounds!(opt, Xlbd )

    function obj(vec,g) 
        out = differential_Xi_opt(i, vec, theta, M, N)
        #verbose ? print(out," ") : ()
        return(out)
    end
    min_objective!(opt,obj)

    (minf,Xi,ret) = optimize(opt,Xi0)
    #println("--The minimum is reached at ",minf, " with ", ret)
    
    return(Xi)
end

function compute_X_opt_slow(theta, M::Para, N::Hyper;
    X0=zeros(N.I,N.J),  stop=1e-3)

    X = copy(X0)
    for i in 1:N.I
        X[i,:] = compute_Xi_opt(i, theta, M, N, Xi0=X0[i,:], stop=stop)
    end
    return(X)
end

######################
### wrap-up
function compute_X_opt(theta, M::Para, N::Hyper;
    X0=zeros(N.I,N.J), Xtol=1e-2)
    
    X = compute_X_opt_fast(theta, M, N, X0=X0,  
            Xtol=Xtol, maxloop = 1e7, step = 0.2)

    if isnan(X[1,1]) | isinf(X[1,1]) 
        X = compute_X_opt_slow(theta, M, N, X0=X0,stop=Xtol/N.J)
    end
    return(X)
end

In [ ]:
function compute_e_opt(X, theta,M::Para,N::Hyper)
    
    e0 = zeros(N.I,N.J)
    e1 = zeros(N.I,N.J,N.J)
    for i in 1:N.I, j1 in 1:N.J
        mj1 = m(theta[j1],M)
        Xk = X[i,j1] - M.k[j1] / q(theta[j1],M)
        e0[i,j1] = e2s(mj1 * Xk ,M)
        for j0 in 1:N.J
            XXk = X[i,j1] - X[i,j0] - M.k[j1] / q(theta[j1],M)
            e1[i,j0,j1] = e2s(M.xi * mj1 * XXk ,M) 
        end
    end
    return(e0,e1)
end

To determine the efficient allocation, we find the objective function to minimize using these four equilibrium functions
\begin{align*}
    & 0=\int_{\mathcal I\times\mathcal J}\xi s(e_{ij_1}^{j_0})\left(\frac{k_{j_1}}{q(\theta_{j_1})}-\eta (X_{ij_1}-X_{ij_0})\right)n_{ij_0}didj_0+\int_\mathcal{I}s(e_{ij_1}^{0})\left(\frac{k_{j_1}}{q(\theta_{j_1})}-\eta X_{ij_1}\right) u_idi
\end{align*}

In [ ]:
function objective_theta_opt(u, n, e0, e1, X, theta, M::Para, N::Hyper)

    res = zeros(N.J)
    for j1 in 1:N.J
        kq = M.k[j1] / q(theta[j1],M)
        for i in 1:N.I
            ## p = 0
            kqX = kq - M.eta * X[i,j1]
            res[j1] += s(e0[i,j1],M) * kqX * u[i]
            
            ## p =j0
            for j0 in 1:N.J
                kqX = kq - M.eta * (X[i,j1] - X[i,j0])
                res[j1] += M.xi * s(e1[i,j0,j1],M) * n[i,j0] * kqX
            end
        end
    end 
    return(sqrt(sum(abs2,res)) / N.J)
end

function compute_optimum(M::Para,N::Hyper ;
    theta0 = fill(0.1,N.J), #A0 = compute_segm_equilibrium(Taxes(0.,N),M,N),
    verbose = false, maxti= 60, Xtol=1e-2, algo=:LN_SBPLX, valf = 1e-3 )   

    ## parameters for optimization
    opt = Opt(algo, length(theta0)) 
  #  xtol_rel!(opt,1e-10)
    stopval!(opt,valf)
    maxtime!(opt,maxti)
    
    lower_bounds!(opt, fill(1e-15,N.J))
    upper_bounds!(opt, fill(Inf,N.J))
    
    function obj(theta,g) 
        X = compute_X_opt(theta,M,N,Xtol=Xtol)
        e0, e1 = compute_e_opt(X,theta,M,N)
        u, n = compute_n(e0, e1, X, theta,M,N)
        out = objective_theta_opt(u, n, e0, e1, X, theta,M,N)
     #   verbose ? print(out, " ") : ()
        return(out)
    end
    min_objective!(opt,obj)

    (minf,theta,ret) =  optimize(opt,theta0)
    verbose ? println("The minimum is reached at ",minf, " with ", ret) : ()
    
    ### alloc output
    X = compute_X_opt(theta,M,N,Xtol=Xtol)
    e0, e1 = compute_e_opt(X,theta,M,N)
    u, n = compute_n(e0, e1, X, theta,M,N)
    return(Alloc(X,e0,e1,theta,u,n))
end


To determine the efficient allocation, we look for the optimal taxes.
First, we must have 
\begin{align}
T_{ij}&=-\frac{\eta}{1-\eta}(X_{ij}-\mathbb{E}_{ip|j}(X_{ij}-X_{ip})),\\
&=T^X X_{ij}+ T_{j}^{f0}.
\end{align}
Then, we must have 
\begin{align}
\Theta^{j_0}_{ij_1}+T_{ij_1}-T_{ij_0}&=-\frac{\eta}{1-\eta}(X_{ij_1}-X_{ij_0}-\mathbb{E}_{ip|j}(X_{ij}-X_{ip})),\\
&\Theta^{j_0}_{ij_1}= \Theta_{j_1}^{f1}.
\end{align}


We fix a Taxes object (uncompensated) with 
\begin{align}
&t^w=-\frac{1-\beta-\eta}{(1-\beta)(1-\eta)}\\
& T^X=-\frac{\eta}{1-\eta}
\end{align}
and
\begin{align*}
    &T_{j_1}^{f0}=\Theta_{j_1}^{f1}=\frac{\eta}{1-\eta}\frac{\int_{\mathcal I\times\mathcal J}\xi s(e_{ij_1}^{j_0})\left(X_{ij_1}-X_{ij_0}\right)n_{ij_0}didj_0+\int_\mathcal{I}s(e_{ij_1}^{0})X_{ij_1} u_idi}{\int_{\mathcal I\times\mathcal J}\xi s(e_{ij_1}^{j_0})n_{ij_0}didj_0+\int_\mathcal{I}s(e_{ij_1}^{0}) u_idi}=\frac{1}{1-\eta}\frac{k_{j_1}}{q(\theta_{j_1})}.
\end{align*}

In [ ]:
function optax(opA::Alloc,M::Para,N::Hyper)
    tauw = -(1-M.beta-M.eta)/((1-M.beta)*(1-M.eta))
    tauX = - M.eta/(1-M.eta)
## I use two ways to compute Tf. They give different results due to numerical errors
    
## version 1
    Tf = zeros(N.J)
    for j1 in 1:N.J
        num=0
        den =0
        for i in 1:N.I
            num += s(opA.e0[i,j1],M) * opA.X[i,j1] * opA.u[i]
            den += s(opA.e0[i,j1],M) * opA.u[i]
            for j0 in 1:N.J
                num += M.xi * s(opA.e1[i,j0,j1],M) * (opA.X[i,j1]-opA.X[i,j0]) * opA.n[i,j0]
                den += M.xi * s(opA.e1[i,j0,j1],M) * opA.n[i,j0]
            end
        end        
        Tf[j1] = M.eta/(1-M.eta) * num / den 
    end
    
## version 2
    Tf2 = [M.k[j]/q(opA.theta[j],M) for j in 1:N.J] ./ (1-M.eta)
    return(Taxes(tauw,0.,tauX,Tf,Tf),  Taxes(tauw,0.,tauX,Tf2,Tf2))
end 
